# 1. Data Crawling

In [ ]:
# Import required Python package
!pip install pandas

# Install Node.js (because tweet-harvest built using Node.js)
!sudo apt-get update
!sudo apt-get install -y ca-certificates curl gnupg
!sudo mkdir -p /etc/apt/keyrings
!curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg

!NODE_MAJOR=20 && echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list

!sudo apt-get update
!sudo apt-get install nodejs -y

!node -v

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,809 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,760 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,103 kB]
Get:13 http://archive.ubuntu.com/ubu

## Gathering Data

In [ ]:
import os
import subprocess
import pandas as pd
from datetime import datetime, timedelta

# Token autentikasi
token = "e28a54897efa45fc8621aa3ee98a29024445fcde"

# Direktori penyimpanan file
output_dir = "tweets-data"

# Pastikan folder tweets-data ada, jika tidak buat
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Fungsi untuk membuat tanggal batch per 1 hari
def generate_daily_batches(start_date, end_date):
    batches = []
    current_date = start_date
    while current_date < end_date:
        next_batch = current_date + timedelta(days=1)
        if next_batch > end_date:
            next_batch = end_date
        batches.append((current_date, next_batch))
        current_date = next_batch + timedelta(days=1)
    return batches

# Scraping per batch
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 6, 20)
batches = generate_daily_batches(start_date, end_date)

total_tweets = 0

for start, end in batches:
    # Tentukan nama file dengan path ke folder tweets-data
    pathFile = os.path.join(output_dir, f'kaburajadulu_{start.strftime("%Y_%m_%d")}_to_{end.strftime("%Y_%m_%d")}.csv')
    filename = f'kaburajadulu_{start.strftime("%Y_%m_%d")}_to_{end.strftime("%Y_%m_%d")}.csv'
    search_keyword = f'kaburajadulu since:{start.strftime("%Y-%m-%d")} until:{end.strftime("%Y-%m-%d")}'
    limit = 10000

    print(f"Scraping: {search_keyword}")
    command = f'npx --yes tweet-harvest@latest -o "{filename}" -s "{search_keyword}" -l {limit} --token "{token}"'
    subprocess.run(command, shell=True)

    try:
        df = pd.read_csv(pathFile)
        num_tweets = len(df)
        print(f"Berhasil mengambil {num_tweets} tweet untuk batch {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}")
        total_tweets += num_tweets
    except Exception as e:
        print(f"Gagal membaca file {pathFile}: {e}")

    # Tambahkan baris kosong untuk spasi antar batch
    print()

print(f"Total tweet yang berhasil diambil (belum deduplikasi): {total_tweets}")

Scraping: kaburajadulu since:2025-01-01 until:2025-01-02
Berhasil mengambil 13 tweet untuk batch 2025-01-01 to 2025-01-02

Scraping: kaburajadulu since:2025-01-03 until:2025-01-04
Gagal membaca file tweets-data/kaburajadulu_2025_01_03_to_2025_01_04.csv: [Errno 2] No such file or directory: 'tweets-data/kaburajadulu_2025_01_03_to_2025_01_04.csv'

Scraping: kaburajadulu since:2025-01-05 until:2025-01-06
Berhasil mengambil 5 tweet untuk batch 2025-01-05 to 2025-01-06

Scraping: kaburajadulu since:2025-01-07 until:2025-01-08
Berhasil mengambil 12 tweet untuk batch 2025-01-07 to 2025-01-08

Scraping: kaburajadulu since:2025-01-09 until:2025-01-10
Gagal membaca file tweets-data/kaburajadulu_2025_01_09_to_2025_01_10.csv: [Errno 2] No such file or directory: 'tweets-data/kaburajadulu_2025_01_09_to_2025_01_10.csv'

Scraping: kaburajadulu since:2025-01-11 until:2025-01-12
Berhasil mengambil 7 tweet untuk batch 2025-01-11 to 2025-01-12

Scraping: kaburajadulu since:2025-01-13 until:2025-01-14
Ber

KeyboardInterrupt: 

In [ ]:
import glob

# Direktori tempat file CSV disimpan
output_dir = "tweets-data"

# Nama file output gabungan
final_output_file = "#KaburAjaDulu.csv"

# Dapatkan semua file CSV dalam direktori
csv_files = glob.glob(os.path.join(output_dir, '*.csv'))

# List untuk menampung DataFrame yang akan digabungkan
df_list = []

# Loop untuk membaca setiap file CSV dan menambahkannya ke dalam list
for file in csv_files:
    try:
        # Membaca file CSV
        df = pd.read_csv(file)
        # Menambahkan DataFrame ke dalam list
        df_list.append(df)
        print(f"Berhasil membaca {file}")
    except Exception as e:
        print(f"Gagal membaca {file}: {e}")

# Menggabungkan semua DataFrame dalam list menjadi satu DataFrame
if df_list:
    final_df = pd.concat(df_list, ignore_index=True)
    # Menyimpan DataFrame gabungan ke file CSV
    final_df.to_csv(final_output_file, index=False)
    print(f"File gabungan disimpan ke {final_output_file}")
else:
    print("Tidak ada file CSV yang ditemukan untuk digabungkan.")
